In [ ]:
# https://www.linkedin.com/pulse/python-finance-part-4-stock-options-henry-meier

In [1]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

# Set stock ticker and expiry date
ticker = input("Enter stock ticker: ")
expiry = input("Enter expiry date: ")

# Get stock data for the past year
start_date = pd.to_datetime('today') - pd.DateOffset(years=1)
end_date = pd.to_datetime('today')
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Calculate historical volatility
returns = stock_data['Adj Close'].pct_change().dropna()
volatility = returns.std() * np.sqrt(252)

# Get option chain for stock ticker and expiry date
option_chain = yf.Ticker(ticker).option_chain(expiry)

# Filter option chain for puts and calls
puts = option_chain.puts.sort_values(by='lastPrice')
calls = option_chain.calls.sort_values(by='lastPrice')

# Filter for conservative trader
conservative_puts = puts[puts["inTheMoney"] == True].head(5)
conservative_calls = calls[calls["inTheMoney"] == True].tail(5)

# Set risk-free interest rate
r = -0.0626

# Calculate time to expiration in years
t = (pd.to_datetime(expiry) - pd.to_datetime('today')).days / 365 

Enter stock ticker:  AMD
Enter expiry date:  2024-02-23


[*********************100%%**********************]  1 of 1 completed


In [2]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

# Define Black-Scholes formula
def black_scholes(S, K, t, r, sigma, option='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    if option == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
    else:
        return K * np.exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Set stock ticker and expiry date
ticker = input("Enter stock ticker: ")
expiry = input("Enter expiry date: ")

# Get stock data for the past year
start_date = pd.to_datetime('today') - pd.DateOffset(years=1)
end_date = pd.to_datetime('today')
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Calculate historical volatility
returns = stock_data['Adj Close'].pct_change().dropna()
volatility = returns.std() * np.sqrt(252)

# Get option chain for stock ticker and expiry date
option_chain = yf.Ticker(ticker).option_chain(expiry)

# Filter option chain for puts and calls
puts = option_chain.puts.sort_values(by='lastPrice')
calls = option_chain.calls.sort_values(by='lastPrice')

# Filter for conservative trader
conservative_puts = puts[puts["inTheMoney"] == True].head(5)
conservative_calls = calls[calls["inTheMoney"] == True].tail(5)

# Set risk-free interest rate
r = -0.0626

# Calculate time to expiration in years
t = (pd.to_datetime(expiry) - pd.to_datetime('today')).days / 365 

# Estimate fair price for each option using Black-Scholes formula
conservative_puts['fair_price'] = black_scholes(
    conservative_puts['lastPrice'], conservative_puts['strike'], t, r, volatility,
    option='put')
conservative_calls['fair_price'] = black_scholes(
    conservative_calls['lastPrice'], conservative_calls['strike'], t, r, volatility,
    option='call')

# Calculate expected return for each option
conservative_puts['expected_return'] = (conservative_puts['fair_price'] - conservative_puts['lastPrice']) / \
                                       conservative_puts['lastPrice']
conservative_calls['expected_return'] = (conservative_calls['fair_price'] - conservative_calls['lastPrice']) / \
                                        conservative_calls['lastPrice']

# Rank options by expected return and suggest top 3 put and call
suggested_puts = conservative_puts.sort_values('expected_return', ascending=False).head(3)
suggested_calls = conservative_calls.sort_values('expected_return', ascending=False).head(3)

# Print suggested options and stock price
market_price = yf.Ticker(ticker).fast_info['lastPrice']
print()
print("Suggested puts:")
print(suggested_puts[['strike', 'lastTradeDate', 'lastPrice', 'ask', 'impliedVolatility', 'fair_price',
                      'expected_return']].to_string(index=False))
print("\nStock Price:")
print(f"The current price of {ticker} is ${market_price:.2f}")

print("\nSuggested calls:")
print(suggested_calls[['strike', 'lastTradeDate', 'lastPrice', 'ask', 'impliedVolatility', 'fair_price',
                       'expected_return']].to_string(index=False))
        

Enter stock ticker:  AMD
Enter expiry date:  2024-02-23


[*********************100%%**********************]  1 of 1 completed



Suggested puts:
 strike             lastTradeDate  lastPrice  ask  impliedVolatility  fair_price  expected_return
  175.0 2024-02-16 20:59:57+00:00       5.76 5.80           0.536137  169.390133        28.408009
  177.5 2024-02-16 20:59:03+00:00       7.30 7.35           0.542241  170.352278        22.335928
  180.0 2024-02-16 20:59:23+00:00       8.98 9.10           0.551518  171.174422        18.061740

Stock Price:
The current price of AMD is $173.87

Suggested calls:
 strike             lastTradeDate  lastPrice   ask  impliedVolatility  fair_price  expected_return
   75.0 2024-02-16 14:58:55+00:00     100.41 99.55           2.523441   25.345657        -0.747578
   80.0 2024-02-15 16:57:49+00:00      95.83 94.80           2.636722   15.761801        -0.835523
   85.0 2024-02-13 15:41:32+00:00      87.47 89.70           2.230473    3.271506        -0.962599
